<a href="https://colab.research.google.com/github/steph2088/code/blob/main/Movie%20reviews%20sentiment%20analysis_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [6]:
import io
import pandas as pd


In [7]:
df = pd.read_csv(io.BytesIO(uploaded['IMDB Dataset.csv']))

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df.shape

(50000, 2)

In [10]:
type(df)  #determine the type of an object

pandas.core.frame.DataFrame

In [11]:
df.tail() # quickly inspecting the last 5 rows of this dataset.

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [12]:
df["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [13]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)  #label  positive -> 1, negative -> 0

In [14]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [15]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [16]:
df["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
train_data, test_data = train_test_split(df, test_size = 0.2, random_state=42)

In [19]:
train_data.shape

(40000, 2)

In [20]:
test_data.shape

(10000, 2)

In [21]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [22]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [23]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [24]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [25]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [26]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [27]:
# LSTM model building
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"]) #The model is compiled with the Adam optimizer, binary crossentropy loss, and accuracy metric.

In [30]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 [==============================] - 402s 798ms/step - loss: 0.4006 - accuracy: 0.8141 - val_loss: 0.3671 - val_accuracy: 0.8288
Epoch 2/5
500/500 [==============================] - 396s 793ms/step - loss: 0.2647 - accuracy: 0.8942 - val_loss: 0.2944 - val_accuracy: 0.8750
Epoch 3/5
500/500 [==============================] - 434s 868ms/step - loss: 0.2193 - accuracy: 0.9156 - val_loss: 0.3013 - val_accuracy: 0.8752
Epoch 4/5
500/500 [==============================] - 407s 812ms/step - loss: 0.1829 - accuracy: 0.9293 - val_loss: 0.3407 - val_accuracy: 0.8779
Epoch 5/5
500/500 [==============================] - 398s 794ms/step - loss: 0.1601 - accuracy: 0.9380 - val_loss: 0.3726 - val_accuracy: 0.8645


In [31]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [33]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 [==============================] - 31s 97ms/step - loss: 0.3487 - accuracy: 0.8678


In [34]:
print(loss)

0.34870630502700806


In [35]:
print(accuracy)

0.8677999973297119


In [36]:
def predictive_system(review): #building predictive system
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [37]:
predictive_system("I thought this movie did a down right good job")

1/1 [==============================] - 1s 531ms/step


'positive'

In [38]:
predictive_system("Bad plot, bad dialogue, bad acting, idiotic")

1/1 [==============================] - 0s 64ms/step


'negative'

In [ ]:
# To improve the model, there are several ways:
# such as enhance on data preprocessing, feature engineering and others.